## Loading the Data Frame

In [1]:
import time
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_1f6f2e6280144b23bb3fd53cbdb85cdd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='H3RY-cRAHjH4W0Kbzwe1nQnJy4oM49dgBZOdz0R3AUwB',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_1f6f2e6280144b23bb3fd53cbdb85cdd.get_object(Bucket='qwerty28b58f4c38184cafbd941800e4d7f669',Key='Collision_Statistics_Monthly_2000-2003.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,﻿ComID,GeogName,DataGroup1,DataGroup1_Order,DataGroup2,Year,Fatal,Injury,Property Damage,Total
0,cnt1201,Shelburne County,Total Reporting,1,Number,2001,3,60,148,211
1,cnt1201,Shelburne County,Total Reporting,1,Number,2002,2,47,122,171
2,cnt1201,Shelburne County,Total Reporting,1,Number,2003,1,69,141,211
3,cnt1201,Shelburne County,Total Reporting,1,Number,2000,0,49,135,184
4,cnt1202,Yarmouth County,Total Reporting,1,Number,2000,2,84,237,323


## Connecting Pandas DataFrame with SQL Spark

In [19]:
time1 = time.time()
sqlContext = SQLContext(sc)
sdf = sqlContext.createDataFrame(df_data_1)
sdf.printSchema()
print("Response Time: %s seconds" % (time.time() - time1))

root
 |-- ﻿ComID: string (nullable = true)
 |-- GeogName: string (nullable = true)
 |-- DataGroup1: string (nullable = true)
 |-- DataGroup1_Order: long (nullable = true)
 |-- DataGroup2: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Fatal: double (nullable = true)
 |-- Injury: double (nullable = true)
 |-- Property Damage: double (nullable = true)
 |-- Total: double (nullable = true)

Response Time: 0.237887859344 seconds


### Problem 4.1 - Capture total incident counts (including Fatal, Injury and Property Damage) in a year (grouped by year). 

In [20]:
time2 = time.time()
sdf.registerTempTable("abc")

abc = sqlContext.sql("SELECT year, Sum(Total) FROM abc group by year order by year")
abc.show()
print("Response Time: %s seconds" % (time.time() - time2))

+----+------------------+
|year|        sum(Total)|
+----+------------------+
|2000| 60824.20000000001|
|2001|           58499.4|
|2002|           55977.2|
|2003|50492.899999999994|
+----+------------------+

Response Time: 1.15971589088 seconds


### Problem 4.2 - Capture sum of injuries in Nova Scotia grouped by year. 

In [21]:
time3 = time.time()
sdf.registerTempTable("abc")

abc = sqlContext.sql("SELECT year, Sum(Injury) FROM abc WHERE GeogName = 'Nova Scotia' group by year order by year")
abc.show()
print("Response Time: %s seconds" % (time.time() - time3))

+----+-----------------+
|year|      sum(Injury)|
+----+-----------------+
|2000|           9848.6|
|2001|8975.499999999998|
|2002|           8471.1|
|2003|7488.099999999999|
+----+-----------------+

Response Time: 0.436430931091 seconds


### Problem 4.3 - Capture sum of injury in Yarmouth Country in the year of 2000.

In [22]:
time4 = time.time()
sdf.registerTempTable("abc")

abc = sqlContext.sql("SELECT year, Sum(Injury) FROM abc WHERE GeogName = 'Yarmouth County' and Year = 2000 group by year ")
abc.show()
print("Response Time: %s seconds" % (time.time() - time4))

+----+-----------+
|year|sum(Injury)|
+----+-----------+
|2000|      174.1|
+----+-----------+

Response Time: 0.544846057892 seconds
